In [1]:
import sys
sys.executable

'd:\\1.VINA\\ai_project\\milvus_ai_local\\.venv\\Scripts\\python.exe'

In [1]:
import pymilvus
print(pymilvus.__version__)

2.6.5


In [3]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
result = sock.connect_ex(("10.50.171.40", 19530))
print("Connection result:", result)

Connection result: 0


In [2]:
import torch
print(torch.__file__)

d:\1.VINA\ai_project\milvus_ai_local\.venv\Lib\site-packages\torch\__init__.py


In [4]:
import os
os.environ["HF_HUB_TIMEOUT"] = "120"

In [17]:
# from sentence_transformers import SentenceTransformer

# Create the embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [9]:
from pymilvus import connections, utility, Collection

connections.connect(
    alias="default",
    host="10.50.171.40",
    port=19530
)

print("Milvus version:", utility.get_server_version())

Milvus version: 2.6.7


In [ ]:
#Read Excel File for sample first

import pandas as pd


df = pd.read_excel(r"D:\\1.VINA\\ai_project\\sample_dataset.xlsx")

df.head()

,Ticket Number,Ticket Summary,Type Ticket,Category,Sub Category,New Category,Month,Created By,Department,Assign Name,Quarter,Troubleshoot Step
0,11549012025,Assign Acc Accrued New Transaction on Dec (Bal...,Yonsuite,Custom Report Yonsuite,Custom Existing Report Yonsuite,Custom Report Yonsuite,January,Dian Annisa Gustamal,FA,Dea Aldiani,Q1,1. Add new COAs in report item \n2. Recalculat...
1,11668022025,missing JIT menu,JIT,Installation JIT,Installation JIT,Installation JIT,February,Pissa Wigiani,PRODUKSI,Dea Aldiani,Q1,1. Check and configure the system date and tim...
2,11366012025,Installation JIT,JIT,Installation JIT,Installation JIT,Installation JIT,January,Kiki Nur Alif,LABORAT,Dea Aldiani,Q1,1. Check and configure the system date and tim...
3,11463012025,installation,JIT,Installation JIT,Installation JIT,Installation JIT,January,Ilman Maulana Firmansyah,PRODSCAN,Dea Aldiani,Q1,1. Check and configure the system date and tim...
4,11487012025,LUPA PASSWORD ORANGE,Orange,Troubleshooting Orange,OTHER ORANGE,Account Access OrangE,January,Enok Nurbayasari,PRODUKSI,Ervina Maharani,Q1,Update new password using upload template


In [21]:
df.drop(columns=['Sub Category', 'Quarter'], inplace=True)
df.head()

,Ticket Number,Ticket Summary,Type Ticket,Category,New Category,Month,Created By,Department,Assign Name,Troubleshoot Step
0,11549012025,Assign Acc Accrued New Transaction on Dec (Bal...,Yonsuite,Custom Report Yonsuite,Custom Report Yonsuite,January,Dian Annisa Gustamal,FA,Dea Aldiani,1. Add new COAs in report item \n2. Recalculat...
1,11668022025,missing JIT menu,JIT,Installation JIT,Installation JIT,February,Pissa Wigiani,PRODUKSI,Dea Aldiani,1. Check and configure the system date and tim...
2,11366012025,Installation JIT,JIT,Installation JIT,Installation JIT,January,Kiki Nur Alif,LABORAT,Dea Aldiani,1. Check and configure the system date and tim...
3,11463012025,installation,JIT,Installation JIT,Installation JIT,January,Ilman Maulana Firmansyah,PRODSCAN,Dea Aldiani,1. Check and configure the system date and tim...
4,11487012025,LUPA PASSWORD ORANGE,Orange,Troubleshooting Orange,Account Access OrangE,January,Enok Nurbayasari,PRODUKSI,Ervina Maharani,Update new password using upload template


In [22]:
df["embed_text"] = (
    "Problem: " + df["Ticket Summary"].fillna("") +
    "\nSolution: " + df["Troubleshoot Step"].fillna("")
)

df.head()

,Ticket Number,Ticket Summary,Type Ticket,Category,New Category,Month,Created By,Department,Assign Name,Troubleshoot Step,embed_text
0,11549012025,Assign Acc Accrued New Transaction on Dec (Bal...,Yonsuite,Custom Report Yonsuite,Custom Report Yonsuite,January,Dian Annisa Gustamal,FA,Dea Aldiani,1. Add new COAs in report item \n2. Recalculat...,Problem: Assign Acc Accrued New Transaction on...
1,11668022025,missing JIT menu,JIT,Installation JIT,Installation JIT,February,Pissa Wigiani,PRODUKSI,Dea Aldiani,1. Check and configure the system date and tim...,Problem: missing JIT menu\nSolution: 1. Check ...
2,11366012025,Installation JIT,JIT,Installation JIT,Installation JIT,January,Kiki Nur Alif,LABORAT,Dea Aldiani,1. Check and configure the system date and tim...,Problem: Installation JIT\nSolution: 1. Check ...
3,11463012025,installation,JIT,Installation JIT,Installation JIT,January,Ilman Maulana Firmansyah,PRODSCAN,Dea Aldiani,1. Check and configure the system date and tim...,Problem: installation\nSolution: 1. Check and ...
4,11487012025,LUPA PASSWORD ORANGE,Orange,Troubleshooting Orange,Account Access OrangE,January,Enok Nurbayasari,PRODUKSI,Ervina Maharani,Update new password using upload template,Problem: LUPA PASSWORD ORANGE\nSolution: Updat...


In [24]:
texts = df["embed_text"].tolist()
# embeddings = model.encode(texts)
embeddings_v2 = model3.encode(texts)

In [25]:
print(embeddings_v2.shape)

(110, 768)


In [19]:
from pymilvus import (
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    utility
)

collection_name = "kb_docs"

if not utility.has_collection(collection_name):
    fields = [
    FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=True
    ),
    FieldSchema(
        name="embedding",
        dtype=DataType.FLOAT_VECTOR,
        dim=384
    ),
    FieldSchema(
        name="ticket_number",
        dtype=DataType.VARCHAR,
        max_length=12
    ),
    FieldSchema(
        name="ticket_summary",
        dtype=DataType.VARCHAR,
        max_length=1024
    ),
    FieldSchema(
        name="type_ticket",
        dtype=DataType.VARCHAR,
        max_length=64
    ),
    FieldSchema(
        name="category",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="new_category",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="department",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="assign_name",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="troubleshoot_step",
        dtype=DataType.VARCHAR,
        max_length=4096
    ),
    ]

    schema = CollectionSchema(
        fields=fields,
        description="Apollo Helpdesk Knowledge Base"
    )

    collection = Collection(
            name=collection_name,
            schema=schema
    )

    print("Collection created:", collection_name)
else:
    collection = Collection(collection_name)
    print("Collection already exists:", collection_name)

Collection already exists: kb_docs


In [32]:
from pymilvus import (
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    utility
)

collection_name = "kb_docs_v2_model3"

if not utility.has_collection(collection_name):
    fields = [
    FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=True
    ),
    FieldSchema(
        name="embedding_v2",
        dtype=DataType.FLOAT_VECTOR,
        dim=768
    ),
    FieldSchema(
        name="ticket_number",
        dtype=DataType.VARCHAR,
        max_length=12
    ),
    FieldSchema(
        name="ticket_summary",
        dtype=DataType.VARCHAR,
        max_length=1024
    ),
    FieldSchema(
        name="type_ticket",
        dtype=DataType.VARCHAR,
        max_length=64
    ),
    FieldSchema(
        name="category",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="new_category",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="department",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="assign_name",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="troubleshoot_step",
        dtype=DataType.VARCHAR,
        max_length=4096
    ),
    ]

    schema = CollectionSchema(
        fields=fields,
        description="Apollo Helpdesk Knowledge Base v2"
    )

    collection = Collection(
            name=collection_name,
            schema=schema
    )

    print("Collection created:", collection_name)
else:
    collection = Collection(collection_name)
    print("Collection already exists:", collection_name)

Collection created: kb_docs_v2_model3


In [ ]:
# utility.drop_collection("kb_docs_v2_model3")

In [ ]:
collection.create_index(
    field_name="embedding_v2",
    index_params={
        "metric_type": "COSINE",
        "index_type": "HNSW",
        "params": {"M": 8, "efConstruction": 64}
    }
)

Status(code=0, message=)

In [34]:
collection.load()

In [ ]:
data_to_insert = [
    embeddings_v2.tolist(),
    df["Ticket Number"].astype(str).tolist(),
    df["Ticket Summary"].tolist(),
    df["Type Ticket"].tolist(),
    df["Category"].tolist(),
    df["sub Category"].astype(str).tolist(),
    df["Department"].tolist(),
    df["Assign Name"].tolist(),
    df["Troubleshoot Step"].tolist()
]

collection.insert(data_to_insert)
collection.flush()

In [ ]:
#Checking

# col = Collection("kb_docs")
col2 = Collection("kb_docs_v2_model3")
print(col.num_entities)
print(col.indexes)

110


In [ ]:
#Testing Model 

query_embedding = model.encode(
    "tidak bisa scan",
    normalize_embeddings=True
)

results = Collection("kb_docs").search(
    data=[query_embedding],
    anns_field="embedding",
    param={"metric_type": "IP", "params": {"ef": 64}},
    limit=5,
    output_fields=["ticket_summary", "troubleshoot_step"]
)

for hit in results[0]:
    print(hit.score, hit.entity.get("troubleshoot_step"))

0.4714670181274414 1. Check and ensure the default browser is correctly configured
2. Check and configure the internet option settings
3. Check and configure the IE mode settings
0.43538516759872437 1. Checking the related device by remote
2. Checking the orangE service performance
0.4254194498062134 1. Check and configure the system date and time settings
2. Check and configure the system language settings
3. Install the JIT application ( if it's not already installed)
4. Create and send the application shortcut on the dekstop
0.4209485650062561 1. Check the user session
2. Kill the active session
3. If don't want to kill the active session, add one more session access
0.40712130069732666 1. Check and ensure the default browser is correctly configured
2. Check and configure the internet option settings
3. Check and configure the IE mode settings


In [ ]:
#Model 2

from sentence_transformers import SentenceTransformer

model2 = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [7]:
from pymilvus import (
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    utility
)

In [10]:
#Testing Model 2

query_embedding = model2.encode(
    "tidak bisa scan",
    normalize_embeddings=True
)

results = Collection("kb_docs").search(
    data=[query_embedding],
    anns_field="embedding",
    param={"metric_type": "IP", "params": {"ef": 64}},
    limit=5,
    output_fields=["ticket_summary", "troubleshoot_step"]
)

for hit in results[0]:
    print(hit.score, hit.entity.get("troubleshoot_step"))

0.32928285002708435 1. Check and configure the system date and time settings
2. Check and configure the system language settings
3. Install the JIT application ( if it's not already installed)
4. Create and send the application shortcut on the dekstop
0.31934335827827454 1. Check and configure the system date and time settings
2. Check and configure the system language settings
3. Install the JIT application ( if it's not already installed)
4. Create and send the application shortcut on the dekstop
0.31407129764556885 1. Check and configure the system date and time settings
2. Check and configure the system language settings
3. Install the JIT application ( if it's not already installed)
4. Create and send the application shortcut on the dekstop
0.3099151849746704 1. Check and configure the system date and time settings
2. Check and configure the system language settings
3. Install the JIT application ( if it's not already installed)
4. Create and send the application shortcut on the d

In [13]:
#Model 3

from sentence_transformers import SentenceTransformer

model3 = SentenceTransformer("all-mpnet-base-v2")

In [39]:
#Testing Model 3

query_embedding = model3.encode(
    "tidak bisa scan",
    normalize_embeddings=True
)

results = Collection("kb_docs_v2_model3").search(
    data=[query_embedding],
    anns_field="embedding_v2",
    param={"metric_type": "IP", "params": {"ef": 64}},
    limit=5,
    output_fields=["ticket_summary", "troubleshoot_step"]
)

for hit in results[0]:
    print(hit.score, hit.entity.get("troubleshoot_step"))

0.47822895646095276 1. Check and ensure the default browser is correctly configured
2. Check and configure the internet option settings
3. Check and configure the IE mode settings
0.38930025696754456 1. Check the user session
2. Kill the active session
3. If don't want to kill the active session, add one more session access
0.36538469791412354 1. Check and ensure the default browser is correctly configured
2. Check and configure the internet option settings
3. Check and configure the IE mode settings
0.36105042695999146 1. Check and configure the system date and time settings
2. Check and configure the system language settings
3. Check and configure wps settings
4. Check JIT version and replace the shortcut if needed
0.336224228143692 1. Check and ensure barcode in lot barcode scan detail menu
2. Check and ensure barcode in NYX
3. Follow up to GOCT SDI (Eric)
